In [32]:
import os
import itertools
import codecs
import re
import datetime
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage
import pylab
from keras import backend as K
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Input, Dense, Activation,Dropout,Flatten
from keras.layers import Reshape, Lambda
from keras.layers.merge import add, concatenate
from keras.models import Model
from keras.layers.recurrent import GRU
from keras.optimizers import SGD
from keras.utils.data_utils import get_file
from keras.preprocessing import image
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import keras.callbacks

In [106]:
class SelfiModel():
    def __init__(self):
        self.model = Sequential()
        
        self.train_datagen = None
        self.train_generator = None
        
        self.test_datagen = None
        self.test_generator=None
        self.validation_generator=None
        
        self.target_size = (64, 64)
        self.batch_size = 20
        self.num_val_samples = 137
        self.validation_step=100
        self.steps = 2295*1.4/self.batch_size
        self.class_mode = 'categorical'
        self.folder = "data/train_set"
        
    def build_model(self):
        input_shape=(64,64,3)
        num_of_class = len(os.listdir(self.folder))
        self.model.add(Conv2D(32, 3, padding='same', activation='relu', input_shape=input_shape))
        self.model.add(Conv2D(32, 3, padding='same', activation='relu'))
        self.model.add(MaxPooling2D(pool_size = (2,2)))
        self.model.add(Dropout(0.25))
        
        self.model.add(Conv2D(64, 3, padding='same', activation='relu'))
        self.model.add(MaxPooling2D(pool_size = (2,2)))
        self.model.add(Dropout(0.5))
        
        self.model.add(Conv2D(128, 3, padding='same', activation='relu'))
        self.model.add(MaxPooling2D(pool_size = (2,2)))
        self.model.add(Dropout(0.4))
        
        self.model.add(Conv2D(256, 3, padding='same', activation='relu'))
        self.model.add(MaxPooling2D(pool_size = (2,2)))
        self.model.add(Dropout(0.3))
        
        self.model.add(Flatten())
        self.model.add(Dense(512, activation='relu'))
        self.model.add(Dropout(0.55))
        self.model.add(Dense(num_of_class, activation='softmax'))
        
        self.model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
        
    def prepare_train_data(self,url):
        self.train_datagen = ImageDataGenerator(rescale = 1./255,
                            featurewise_std_normalization=True,  # divide inputs by std of the dataset
                            samplewise_std_normalization=True,  # divide each input by its std
                            rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
                            zoom_range = 0.1, # Randomly zoom image 
                            horizontal_flip=True,  # randomly flip images
                            vertical_flip=True)  # randomly flip images
        
        self.train_generator = self.train_datagen.flow_from_directory(url,
                                                                        target_size = self.target_size,
                                                                        color_mode = 'rgb',
                                                                        batch_size = self.batch_size,
                                                                        class_mode = self.class_mode)
        
    def prepare_validation_data(self,url):
        self.test_datagen = ImageDataGenerator(rescale = 1./255)
        self.validation_generator = self.test_datagen.flow_from_directory(url,
                                                                            target_size = self.target_size,
                                                                            color_mode = 'rgb',
                                                                            batch_size = self.batch_size,
                                                                            class_mode = self.class_mode)
        
    def prepare_test_data(self,url):
        self.test_generator = self.test_datagen.flow_from_directory(url,
                                                                      target_size = self.target_size,
                                                                      color_mode = 'rgb',
                                                                      batch_size = 1,
                                                                      class_mode = self.class_mode,
                                                                      shuffle = False)
    
    def train_model(self,epochs=10):
#         self.num_val_samples/self.batch_size
        self.model.fit_generator(self.train_generator,
                                    steps_per_epoch = self.steps,
                                    epochs = epochs,
                                    validation_data = self.validation_generator,
                                    validation_steps = self.validation_step)
    
    def save_model(self):
        self.model.save_weights('weights.h5')
    
    def load_model(self):
        self.model.load_weights("weights.h5")
    
    def predict_class(self,img):
        face = cv2.resize(cv2.imread(img),self.target_size)
        input_data = face.reshape((-1, face.shape[0], face.shape[1], face.shape[2]))
        ans = self.model.predict(input_data)
        mx = -1
        pos= -1
        for i in range(len(ans[0])):
            print(ans[0][i])
            if(mx<ans[0][i]):
                mx=ans[0][i]
                pos= i 
        print(pos)
        plt.imshow(face)
        

In [107]:
if __name__ == "__main__":
    sm = SelfiModel()
    sm.build_model()
#     sm.prepare_train_data('data/train_set')
#     sm.prepare_validation_data('data/val_set')
#     sm.prepare_test_data('data/test_set')

In [108]:
sm.train_model(100)

In [109]:
sm.load_model()